# WEC AC Injection

### Imports

In [1]:
import sys
sys.path.append('../wec-grid-code') # 
import wec_grid_class as wg
import sqlite3
import pandas as pd

    Sets PSSE environment to latest minor version among installed versions of PSSE 35.
    Use psse35.set_minor(n) to set PSSE35 minor version (n) to use.
        Example, for PSSE 35.0.x, use this as: psse35.set_minor(0)


In [15]:
# Read in case file 
case14 = r"../input_files/RTS96DYN/IEEE_24_bus.RAW"

### run for DC 

In [ ]:
# initalize WEC GRID Object
pf = wg.Wec_grid(case14,"DC",3)

### run for AC 

In [16]:
# initalize WEC GRID Object
pf = wg.Wec_grid(case14,"fnsl",3)


 Reading IC, SBASE, REV, XFRRAT, NXFRAT, BASFRQ...

 Converting RAW data from PSS(R)E 33 to PSS(R)E 34 format
 Starting processing of Case identification data records
 Finished processing of 3 records for Case identification data
 Starting processing of Bus data records
 Finished processing of 24 records for Bus data
 Starting processing of Load data records
 Finished processing of 16 records for Load data
 Starting processing of Fixed shunt data records
 Finished processing of 0 records for Fixed shunt data
 Starting processing of Generator data records
 Finished processing of 11 records for Generator data
 Starting processing of Branch data records
 Finished processing of 32 records for Branch data
 Finished processing of 0 records for System switching device data
 Starting processing of Transformer data records
 Finished processing of 24 records for 6 Transformer data
 Starting processing of Area data records
 Finished processing of 1 records for Area data
 Starting processing of T

### Run WEC Model via matlab (takes a minute)

In [ ]:
#pf.run_WEC_Sim()

In [17]:
pf.dataframe.iloc[[0,1,12]]

,BUS_ID,Bus,BASE,PU,ANGLED,Type,P Gen,Q Gen,P Load,Q Load,P,Q
0,1,BUS 1,138.0,1.000000,0.000000,3,35.930069,180.488586,NaN,NaN,35.930069,180.488586
1,2,BUS 2,138.0,0.997822,0.010373,2,67.000000,0.000000,97.0,20.000000,-30.000000,-20.000000
12,13,BUS 13,230.0,0.836927,15.450583,2,200.000000,0.000000,130.0,27.000002,70.000000,-27.000002


### Pull WEC injection values via SQL database

In [18]:
con = sqlite3.connect("../input_files/r2g_database.db")
injection = pd.read_sql_query("SELECT * from WEC_output", con)
print(injection)

      time  ibus         pg   vs         pt   pb         qt         qb
0      0.0   1.0   0.000000  1.0   0.000000  0.0   0.000000   0.000000
1    450.0   1.0  20.284200  1.1  20.284200  0.0  22.064601 -22.064601
2    750.0   1.0  21.086833  1.1  21.086833  0.0  21.173366 -21.173366
3   1050.0   1.0  17.886427  1.1  17.886427  0.0  23.649517 -23.649517
4   1350.0   1.0  14.244265  1.1  14.244265  0.0  26.306681 -26.306681
5   1650.0   1.0  21.451810  1.1  21.451810  0.0  20.855884 -20.855884
6   1950.0   1.0  19.953674  1.1  19.953674  0.0  22.165508 -22.165508
7   2250.0   1.0  17.727287  1.1  17.727287  0.0  24.159891 -24.159891
8   2550.0   1.0  13.796929  1.1  13.796929  0.0  26.609123 -26.609123
9   2850.0   1.0  13.583606  1.1  13.583606  0.0  26.734918 -26.734918
10  3150.0   1.0  15.860482  1.1  15.860482  0.0  25.432033 -25.432033
11  3450.0   1.0  17.103224  1.1  17.103224  0.0  24.629206 -24.629206


In [19]:
wecs = [2,13]
p = [20.284199882370412, 20.284199882370412]
v = [1.1, 1.1]
pf.ac_injection(wecs, p, v, pf.solver, injection.iloc[i].time)
pf.dataframe.iloc[[0,1,12]]


 Power flow data changed for machine "1" at bus 2 [2           138.00]:
 X--ORIGINAL--X  X-NEW VALUE--X  DATA ITEM
    67.0000         20.2842      PG

 Power flow data changed for bus 2 [2           138.00]:
 X--ORIGINAL--X  X-NEW VALUE--X  DATA ITEM
   0.997822         1.10000      VM

 Power flow data changed for machine "1" at bus 13 [13          230.00]:
 X--ORIGINAL--X  X-NEW VALUE--X  DATA ITEM
    200.000         20.2842      PG

 Power flow data changed for bus 13 [13          230.00]:
 X--ORIGINAL--X  X-NEW VALUE--X  DATA ITEM
   0.836927         1.10000      VM

  ITER       DELTAP      BUS         DELTAQ      BUS        DELTA/V/      BUS       DELTAANG      BUS
   0        40.0802(      2     )   70.3702(      2     )
                                                             0.87250(     12     )   0.81465(     21     )
   1         5.4394(     23     )    8.1603(     12     )
                                                             0.23989(     12     )   0.30893( 

,BUS_ID,Bus,BASE,PU,ANGLED,Type,P Gen,Q Gen,P Load,Q Load,P,Q
0,1,BUS 1,138.0,1.000000,0.000000,3,256.338959,121.615013,NaN,NaN,256.338959,121.615013
1,2,BUS 2,138.0,0.997222,-0.090895,2,20.284201,0.000000,97.0,20.000000,-76.715799,-20.000000
12,13,BUS 13,230.0,0.838020,2.771837,2,20.284201,0.000000,130.0,27.000002,-109.715799,-27.000002


In [13]:
for i in range(len(injection)):
    print("Time: {}".format(injection.iloc[i].time))
    print("P setpoint: {}".format(injection.pg.iloc[i]))
    if pf.solver == 'AC':
        pf.ac_injection(wecs, p, v, pf.solver, injection.iloc[i].time)
    if pf.solver == 'DC':
        pf.dc_injection(pf.wecBus_num, injection.pg.iloc[i], pf.solver, injection.iloc[i].time)
    print(pf.dataframe.iloc[[0,1,2]])
    print("========================")

Time: 0.0
P setpoint: 0.0
   BUS_ID    Bus   BASE        PU     ANGLED  Type      P Gen       Q Gen  \
0       1  BUS 1  138.0  1.000000   0.000000     3  35.930069  180.488586   
1       2  BUS 2  138.0  0.997822   0.010373     2  67.000000    0.000000   
2       3  BUS 3  138.0  0.863703  10.755864     1        NaN         NaN   

   P Load  Q Load          P           Q  
0     NaN     NaN  35.930069  180.488586  
1    97.0    20.0 -30.000000  -20.000000  
2    90.0    19.0 -90.000000  -19.000000  
Time: 450.0
P setpoint: 20.284199882370412
   BUS_ID    Bus   BASE        PU     ANGLED  Type      P Gen       Q Gen  \
0       1  BUS 1  138.0  1.000000   0.000000     3  35.930069  180.488586   
1       2  BUS 2  138.0  0.997822   0.010373     2  67.000000    0.000000   
2       3  BUS 3  138.0  0.863703  10.755864     1        NaN         NaN   

   P Load  Q Load          P           Q  
0     NaN     NaN  35.930069  180.488586  
1    97.0    20.0 -30.000000  -20.000000  
2    90.0   

In [ ]:
pf.dataframe.iloc[[1,2]]

In [ ]:
pf.history

In [ ]:
pf.plotSwingBus()

In [ ]:
injection